In [25]:
import pickle
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import (
    ColumnDriftMetric, 
    DatasetDriftMetric, 
    DatasetMissingValuesMetric
)

In [2]:
source_path = '../data/raw/green_tripdata_2023-03.parquet'

In [15]:
df = pd.read_parquet(source_path)

In [16]:
df.shape[0]

72044

In [17]:
df.head(5)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-03-01 00:25:10,2023-03-01 00:35:47,N,1.0,82,196,1.0,2.36,13.5,1.0,0.5,0.00,0.0,NaN,1.0,16.00,2.0,1.0,0.00
1,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,-6.5,-1.0,-0.5,0.00,0.0,NaN,-1.0,-9.00,3.0,1.0,0.00
2,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,6.5,1.0,0.5,0.00,0.0,NaN,1.0,9.00,3.0,1.0,0.00
3,2,2023-02-28 22:59:46,2023-02-28 23:08:38,N,1.0,166,74,1.0,1.66,11.4,1.0,0.5,2.78,0.0,NaN,1.0,16.68,1.0,1.0,0.00
4,2,2023-03-01 00:54:03,2023-03-01 01:03:14,N,1.0,236,229,1.0,3.14,15.6,1.0,0.5,4.17,0.0,NaN,1.0,25.02,1.0,1.0,2.75


In [21]:
with open('../models/model.bin', 'rb') as f:
    dv, model = pickle.load(f)

In [23]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [26]:
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features
)

In [27]:
column_mapping

ColumnMapping(target=None, prediction='prediction', datetime='datetime', id=None, numerical_features=['passenger_count', 'trip_distance', 'fare_amount', 'total_amount'], categorical_features=['PULocationID', 'DOLocationID'], datetime_features=None, target_names=None, task=None, pos_label=1, text_features=None, embeddings=None, user_id='user_id', item_id='item_id', recommendations_type=<RecomType.SCORE: 'score'>)

In [28]:
report = Report(metrics=[ColumnDriftMetric(column_name='prediction'),
                        DatasetDriftMetric(),
                        DatasetMissingValuesMetric()])